# Import Tools

In [17]:
import gc
import os
import random
import warnings
import numpy as np
import pandas as pd
from IPython.display import display
import timm
import torch
import torch.nn as nn  
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
warnings.filterwarnings('ignore', category=Warning)
gc.collect()

0

In [18]:
class Config:
    seed=42
    image_transform=transforms.Resize((512, 512))
    num_folds=5
    root_path = ""
    
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
set_seed(Config.seed)

# Load models

## Resnet34d

In [19]:
models = []
for i in range(Config.num_folds):
    model_resnet = timm.create_model('resnet34d', pretrained=False, num_classes=6, in_chans=1)
    model_resnet.load_state_dict(torch.load(f'{Config.root_path}resnet34d_fold{i}.pth', map_location=torch.device('cpu')))
    models.append(model_resnet)
gc.collect()

49

## Efficientnet

In [20]:
for j in range(Config.num_folds):
    model_effnet = timm.create_model('efficientnet_b0', pretrained=False, num_classes=6, in_chans=1)
    model_effnet.load_state_dict(torch.load(f'{Config.root_path}efficientnet_b0_fold{j}.pth', map_location=torch.device('cpu')))
    models.append(model_effnet)
gc.collect()

45

In [21]:
test_df = pd.read_csv(f"{Config.root_path}test.csv")
submission = pd.read_csv(f"{Config.root_path}sample_submission.csv")
submission = submission.merge(test_df, on='eeg_id', how='left')
submission['path'] = submission['spectrogram_id'].apply(lambda x: f"{Config.root_path}test_spectrograms/{x}.parquet")
display(submission.head())
gc.collect()

,eeg_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,spectrogram_id,patient_id,path
0,3911565283,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,853520,6885,test_spectrograms/853520.parquet


0

In [ ]:
weight_resnet = 0.32
weight_effnet = 0.68
paths = submission['path'].values
test_preds = []

for path in paths:
    eps = 1e-6

    # Read and preprocess spectrogram data
    data = pd.read_parquet(path)
    data = data.fillna(-1).values[:, 1:].T
    data = np.clip(data, np.exp(-6), np.exp(10))
    data = np.log(data)
    
    # Normalize the data
    data_mean = data.mean(axis=(0, 1))
    data_std = data.std(axis=(0, 1))
    data = (data - data_mean) / (data_std + eps)
    data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
    data = Config.image_transform(data_tensor)

    test_pred = []
    
    for model in models:
        model.eval()
        with torch.no_grad():
            pred = F.softmax(model(data.unsqueeze(0)))[0]
            pred = pred.detach().cpu().numpy()
        test_pred.append(pred)
        
    weighted_pred = weight_resnet * np.mean(test_pred[:len(models)//2], axis=0) + \
                     weight_effnet * np.mean(test_pred[len(models)//2:], axis=0)
    
    test_preds.append(weighted_pred)
test_preds = np.array(test_preds)
gc.collect()

In [ ]:
submission = pd.read_csv(f"{Config.root_path}sample_submission.csv")
labels = ['seizure', 'lpd', 'gpd', 'lrda', 'grda', 'other']
for i in range(len(labels)):
    submission[f'{labels[i]}_vote'] = test_preds[:, i]

submission.to_csv("submission.csv", index=None)
display(submission.head())
gc.collect()